# 📊 Báo cáo: Dự đoán xu hướng từ dữ liệu mạng xã hội (hashtags, chủ đề)
---
**Môn học**: Khai thác Dữ liệu\  
**Sinh viên thực hiện**: NHOM 12\  
**Mục tiêu**: Phân tích hashtag, phân cụm chủ đề và dự đoán xu hướng từ dữ liệu mạng xã hội.


## 1. Giới thiệu
Mạng xã hội là nơi liên tục xuất hiện các xu hướng mới (trending topics). Việc phát hiện và dự đoán các xu hướng này từ hashtag và nội dung bài viết là bài toán quan trọng trong lĩnh vực khai thác dữ liệu.

Đề tài này sử dụng dữ liệu Twitter, phân tích các hashtag phổ biến, phân cụm các chủ đề và áp dụng mô hình học máy để dự đoán xu hướng.


In [ ]:
# 2. Thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


## 3. Đọc dữ liệu
Dữ liệu được thu thập từ Twitter và lưu dưới dạng CSV.


In [ ]:
# Đọc dữ liệu
data = pd.read_csv('../data/raw_twitter_data.csv')
data.head()

## 4. Phân tích các Hashtag phổ biến
Trích xuất và thống kê các hashtag xuất hiện nhiều nhất trong dữ liệu.

In [ ]:

# Tách và đếm hashtag
hashtag_list = [tag for sublist in data['hashtags'].dropna().apply(eval) for tag in sublist]
hashtag_counts = pd.Series(hashtag_list).value_counts().head(20)
hashtag_counts.plot(kind='bar', figsize=(12,6), title='Top 20 Hashtag Phổ Biến')
plt.ylabel('Số lần xuất hiện')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


## 5. Phân cụm các chủ đề bằng KMeans
Sử dụng TF-IDF để vector hóa văn bản, sau đó áp dụng KMeans để phân cụm các bài viết theo chủ đề.

In [ ]:

# Vector hóa nội dung bằng TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X_vectors = vectorizer.fit_transform(data['text'].fillna(''))

# Phân cụm KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster'] = kmeans.fit_predict(X_vectors)

# Giảm chiều dữ liệu bằng PCA để trực quan hóa
pca = PCA(n_components=2)
reduced = pca.fit_transform(X_vectors.toarray())
data['PC1'], data['PC2'] = reduced[:,0], reduced[:,1]

# Vẽ biểu đồ phân cụm
fig = px.scatter(data, x='PC1', y='PC2', color=data['cluster'].astype(str),
                 title="Phân cụm chủ đề bằng KMeans + PCA")
fig.show()


## 6. Xây dựng mô hình dự đoán xu hướng
Giả lập đặc trưng và nhãn để huấn luyện mô hình Random Forest.

In [ ]:

# Tạo dữ liệu đặc trưng giả lập (hoặc thay bằng đặc trưng thật nếu có)
X = np.random.rand(len(data), 10)
y = np.random.randint(0, 2, len(data))  # 0: không trend, 1: trend

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


## 7. Đánh giá mô hình
Sử dụng các chỉ số: Accuracy, Precision, Recall, F1-score và ma trận nhầm lẫn.

In [ ]:

# Đánh giá mô hình
print("Báo cáo phân loại:
", classification_report(y_test, y_pred))
print("Ma trận nhầm lẫn:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.show()


## 8. Kết luận & Hướng phát triển
- Dữ liệu mạng xã hội chứa nhiều thông tin hữu ích về xu hướng thông qua hashtag.
- Việc kết hợp phân cụm và mô hình học máy giúp khám phá và dự đoán các chủ đề đang nổi bật.
- Trong tương lai có thể mở rộng bằng cách:
  - Kết hợp dữ liệu thời gian (temporal trends)
  - Phân tích mạng lưới người dùng
  - Cải thiện đặc trưng đầu vào bằng embedding (BERT, Word2Vec...)